In [ ]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# Define constants
c = 2.998e8
e = 1.602e-19
m_e = 9.109e-31
nm = 1e-9

# Generate plasma parameters
def generate_parameters(n_0):
    k_p = np.sqrt(4.0 * np.pi * e**2 * n_0 * 1e6 / (m_e * c**2))
    omega_p = np.sqrt(4.0 * np.pi * e**2 * n_0 * 1e6 / m_e)
    lambda_p = 2.0 * np.pi * c / omega_p
    E0 = c * m_e * omega_p / e
    return k_p, omega_p, lambda_p, E0

# Define the wakefield plotting function
def plot_wakefield(omega0=15, a01=1, tau1=1, tau_delay1=0, a02=0.5, tau2=1, tau_delay2=1):
    n_0 = 1.0e17
    k_p, omega_p, lambda_p, E0 = generate_parameters(n_0)
    tau_delay1 *= 2.0 * np.pi
    tau_delay2 *= 2.0 * np.pi

    # Calculate coordinates
    xi = np.linspace(-lambda_p, 4.0 * lambda_p, 1000)
    psi = k_p * xi
    psi_plot = -psi / (2.0 * np.pi) + k_p * tau_delay1 / (2.0 * np.pi)

    # Define laser vector potentials
    def a_1(psi):
        return np.real(
            a01
            * np.exp(-2.0 * np.log(2) * ((psi - tau_delay1) ** 2) / (tau1**2))
            * np.exp(-1j * omega0 * psi)
        )

    def a_2(psi):
        return np.real(
            a02
            * np.exp(-2.0 * np.log(2) * ((psi - (tau_delay1 + tau_delay2)) ** 2) / (tau2**2))
            * np.exp(-1j * omega0 * psi)
        )

    # Define wake potential
    phi = np.sin(psi * omega0 / 15) * np.exp(-a01 * psi / lambda_p)

    # Create subplots using Plotly
    fig = go.Figure()

    # Plot potential
    fig.add_trace(go.Scatter(x=psi_plot, y=phi, mode="lines", name="Potential (phi)", line=dict(color="blue")))

    # Plot laser vector potentials
    fig.add_trace(go.Scatter(x=psi_plot, y=a_1(psi), mode="lines", name="a_1", line=dict(color="orange", dash="dash")))
    fig.add_trace(go.Scatter(x=psi_plot, y=a_2(psi), mode="lines", name="a_2", line=dict(color="green", dash="dash")))

    # Update layout
    fig.update_layout(
        title="Wakefield Simulation",
        xaxis_title="xi / lambda_p",
        yaxis_title="Field Values",
        legend_title="Legend",
        template="plotly_white",
    )

    # Save to HTML
    fig.write_html("wakefield_simulation.html")
    print("Plot saved as wakefield_simulation.html")

# Call the function to generate the plot
plot_wakefield()

interactive(children=(IntSlider(value=15, description='omega0', max=50, min=15), FloatSlider(value=1.0, descri…